# Prepare mitosis time series data

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from cellpose import models
from cellpose.io import imread
import glob
from pathlib import Path
from PIL import Image, ImageSequence
from tqdm import tqdm
import os
import os.path
# from livecellx import segment
from livecellx import core
from livecellx.core import datasets
from livecellx.core.datasets import LiveCellImageDataset, SingleImageDataset
from skimage import measure
from livecellx.core import SingleCellTrajectory, SingleCellStatic

In [2]:
from livecellx.track.classify_utils import load_class2samples_from_json_dir, load_all_json_dirs
# sample_json_dir = Path("./EBSS_starvation_24h_xy16_annotation")

sample_json_dirs_v0 = [Path(r"./datasets/test_scs_EBSS_starvation/XY1/annotations"), Path(r"./datasets/test_scs_EBSS_starvation/XY16/annotations")]

round1_json_dirs = sample_json_dirs_v0 + [
    Path(r"D:\LiveCellTracker-dev\datasets\mitosis-annotations-2023\shiman_XY01\XY01"),
Path(r"D:\LiveCellTracker-dev\datasets\mitosis-annotations-2023\shiman_XY09\XY09"),
Path(r"D:\LiveCellTracker-dev\datasets\mitosis-annotations-2023\shiman_XY10\XY10"),
Path(r"D:\LiveCellTracker-dev\datasets\mitosis-annotations-2023\Yajushi\tifs_CFP_A549-VIM_lessThan24hr_NoTreat_NA_YL_Ti2e_2022-10-19\XY1\annotations"),
]

round2_json_dirs = [
Path(r"../datasets/mitosis-annotations-2023/shiman_CXA_high_density/C0.5^4/"),
Path(r"../datasets/mitosis-annotations-2023/shiman_CXA_high_density/C0.75^4/"),
Path(r"../datasets/mitosis-annotations-2023/shiman_CXA_high_density/C10^3/"),
Path(r"../datasets/mitosis-annotations-2023/shiman_CXA_high_density/C10^4/")
] + [
    Path(f"../datasets/mitosis-annotations-2023/Gaohan_tifs_CFP_A549-VIM_lessThan24hr_NoTreat_NA_YL_Ti2e_2022-10-19/XY{pos}/annotations") for pos in range(4, 14)
]

sample_json_dirs = sample_json_dirs_v0 + round1_json_dirs + round2_json_dirs
all_class2samples, all_class2sample_extra_info = load_all_json_dirs(sample_json_dirs)
           

{'mitosis': [[SingleCellStatic(id=81aba36a-5344-49ba-9b0f-200441080359, timeframe=258, bbox=[ 768. 1176.  864. 1283.]), SingleCellStatic(id=ab9f97b2-5357-4a4e-a92a-b1cbac3fbf37, timeframe=259, bbox=[ 768. 1183.  860. 1292.]), SingleCellStatic(id=5e013815-e531-4faf-ac11-973164e539d8, timeframe=260, bbox=[ 764. 1190.  865. 1282.]), SingleCellStatic(id=8f2c6601-d68f-4470-b3a6-a7d96ff99ec9, timeframe=261, bbox=[ 760. 1186.  880. 1275.]), SingleCellStatic(id=282f4f4a-bc2b-413a-a35a-89d2afb6302f, timeframe=262, bbox=[ 764. 1187.  876. 1270.]), SingleCellStatic(id=44ee2050-70de-4004-b7bb-6fc27936216d, timeframe=263, bbox=[ 768. 1187.  881. 1267.]), SingleCellStatic(id=1e211900-517c-46b0-aa89-16d556af099a, timeframe=264, bbox=[ 773. 1186.  883. 1264.]), SingleCellStatic(id=fa047f98-9338-4a2e-a26e-12aa1a02fdce, timeframe=265, bbox=[ 822. 1196.  887. 1265.]), SingleCellStatic(id=2466ce33-c249-4d32-a269-db58eaeb31c5, timeframe=265, bbox=[ 756. 1174.  824. 1234.]), SingleCellStatic(id=a2f9bc66-0d8

In [3]:
# all_class2samples, all_class2sample_extra_info

In [4]:
len(all_class2samples["mitosis"]), len(all_class2samples["apoptosis"]), len(all_class2samples["normal"])

(198, 0, 0)

In [5]:
num_zero_len_samples = 0
for key in all_class2samples.keys():
    for sample in all_class2samples[key]:
        if len(sample) == 0:
            num_zero_len_samples += 1
print("num_zero_len_samples: ", num_zero_len_samples)

num_zero_len_samples:  1


Automatically prepare normal samples

require tracking done

In [6]:
# get all scs from class_samples not in normal class
exclude_scs = []
total_non_normal_samples = 0
for class_name, samples in all_class2samples.items():
    if class_name != "normal":
        for sample in samples:
            exclude_scs.extend(sample)
            total_non_normal_samples += 1

exclude_scs = set(exclude_scs)
exclude_scs_ids = {str(sc.id) for sc in exclude_scs}

In [7]:
# from livecellx.core.sct_operator import create_scs_edit_viewer
# sct_operator = create_scs_edit_viewer(exclude_scs, img_dataset = list(exclude_scs)[0].img_dataset)

load all single cells, including mitosis and normal ones, for further generating normal samples automatically

In [8]:
import json
from livecellx.core.single_cell import SingleCellTrajectoryCollection
from livecellx.track.sort_tracker_utils import (
    track_SORT_bbox_from_scs
)

all_scs_json_path = ["./datasets/test_scs_EBSS_starvation/XY1/single_cells.json", "./datasets/test_scs_EBSS_starvation/XY16/single_cells.json"]
# all_scs_json_path = "./datasets/test_scs_EBSS_starvation/XY16/tmp_corrected_scs.json"
sctc = SingleCellTrajectoryCollection()
for json_path in all_scs_json_path:
    print("json path:", json_path)
    _scs = SingleCellStatic.load_single_cells_json(json_path)
    tmp_sctc = track_SORT_bbox_from_scs(_scs, raw_imgs=_scs[0].img_dataset, min_hits=3, max_age=3)
    tids = set(sctc.get_all_tids())
    if len(tids) != 0:
        max_tid = max(tids)
    else:
        max_tid = 0
    for tid, traj in tmp_sctc:
        traj.meta["src_dir"] = json_path
        traj.track_id = tid + max_tid + 1
        sctc.add_trajectory(traj)
        traj_scs = traj.get_all_scs()
        for sc in traj_scs:
            sc.meta["src_dir"] = json_path
    del tmp_sctc

all_scs = SingleCellStatic.load_single_cells_jsons(all_scs_json_path)

json path: ./datasets/test_scs_EBSS_starvation/XY1/single_cells.json
json path: ./datasets/test_scs_EBSS_starvation/XY16/single_cells.json


In [9]:

# with open("./EBSS_starvation_24h_xy16_annotation/single_cell_trajectory_collection.json", "r") as file:
#     json_dict = json.load(file)
# sctc = SingleCellTrajectoryCollection().load_from_json_dict(json_dict)


In [10]:
# set numpy seed
seed = 0
np.random.seed(seed)

objective_sample_num = total_non_normal_samples * 10

normal_frame_len_range = (3, 10)
counter = 0
normal_samples = []
normal_samples_extra_info = []
max_trial_counter = 100000
while counter < objective_sample_num and max_trial_counter > 0:
    # randomly select a sct from sctc
    # generate a list of scs
    track_id = np.random.choice(list(sctc.track_id_to_trajectory.keys()))  
    sct = sctc.get_trajectory(track_id)
    # randomly select a length
    frame_len = np.random.randint(*normal_frame_len_range)
    # generate a sample
    times = list(sct.timeframe_to_single_cell.keys())
    times = sorted(times)
    if len(times) <= frame_len:
        continue
    start_idx = np.random.randint(0, len(times) - frame_len)
    start_time = times[start_idx]
    end_time = times[start_idx + frame_len - 1]

    sub_sct = sct.subsct(start_time, end_time)

    is_some_sc_in_exclude_scs = False
    for time, sc in sub_sct.timeframe_to_single_cell.items():
        # print("sc.id:", sc.id, type(sc.id))
        if str(sc.id) in exclude_scs_ids:
            is_some_sc_in_exclude_scs = True
            break
    if is_some_sc_in_exclude_scs:
        print("some sc in the exclude scs list")
        continue
    
    new_sample = []
    for time, sc in sub_sct.timeframe_to_single_cell.items():
        new_sample.append(sc)
    normal_samples.append(new_sample)
    normal_samples_extra_info.append({"src_dir": sub_sct.get_all_scs()[0].meta["src_dir"]})
    counter += 1
    max_trial_counter -= 1

normal_samples[:2]

some sc in the exclude scs list
some sc in the exclude scs list
some sc in the exclude scs list
some sc in the exclude scs list
some sc in the exclude scs list
some sc in the exclude scs list
some sc in the exclude scs list
some sc in the exclude scs list
some sc in the exclude scs list
some sc in the exclude scs list
some sc in the exclude scs list
some sc in the exclude scs list
some sc in the exclude scs list
some sc in the exclude scs list
some sc in the exclude scs list
some sc in the exclude scs list
some sc in the exclude scs list
some sc in the exclude scs list
some sc in the exclude scs list
some sc in the exclude scs list
some sc in the exclude scs list
some sc in the exclude scs list
some sc in the exclude scs list
some sc in the exclude scs list
some sc in the exclude scs list
some sc in the exclude scs list
some sc in the exclude scs list
some sc in the exclude scs list
some sc in the exclude scs list
some sc in the exclude scs list
some sc in the exclude scs list
some sc 

[[SingleCellStatic(id=056b6e7b-d2cb-4501-b3d2-924e9d149e74, timeframe=73, bbox=[417 387 601 585]),
  SingleCellStatic(id=c6f083e2-fff1-47d9-a2b0-4feff712626b, timeframe=74, bbox=[421 392 603 580]),
  SingleCellStatic(id=3b10c5f7-966a-4282-a661-78da785fff0e, timeframe=75, bbox=[416 391 595 579]),
  SingleCellStatic(id=0816119c-6bdc-49cb-b857-0d43cbb03e61, timeframe=76, bbox=[420 390 600 586]),
  SingleCellStatic(id=2e83a2af-01bc-4453-9f45-49a3c0e1d975, timeframe=77, bbox=[437 402 601 576]),
  SingleCellStatic(id=34b6863b-baf4-4197-9554-f701711c59db, timeframe=78, bbox=[438 399 595 568]),
  SingleCellStatic(id=6ba2fc6f-9c02-4c7b-beda-3fa7a0a1e84a, timeframe=79, bbox=[444 406 595 581]),
  SingleCellStatic(id=97536b78-d230-4c98-9d79-ccac89189aed, timeframe=80, bbox=[443 406 595 576])],
 [SingleCellStatic(id=5e9559bc-f211-4481-b005-fb4ff850ed40, timeframe=173, bbox=[1302 1134 1403 1229]),
  SingleCellStatic(id=174a1612-6faa-4700-a02f-7ae1a3eecb50, timeframe=174, bbox=[1228 1107 1404 1233]),

In [11]:
all_class2samples["normal"].extend(normal_samples)
all_class2sample_extra_info["normal"].extend(normal_samples_extra_info)

In [12]:
len(all_class2samples["normal"]), len(all_class2sample_extra_info["normal"])

(1980, 1980)

## Prepare videos and annotations for MMDetection

In [13]:
classes = all_class2samples.keys()
classes

dict_keys(['mitosis', 'apoptosis', 'normal'])

In [14]:
from livecellx.core.utils import gray_img_to_rgb, rgb_img_to_gray
from livecellx.preprocess.utils import normalize_img_to_uint8

In [15]:
from livecellx.track.classify_utils import video_frames_and_masks_from_sample, combine_video_frames_and_masks

In [16]:
from typing import List
import cv2
import numpy as np
import pandas as pd

from livecellx.core.sc_video_utils import gen_mp4_from_frames, gen_class2sample_samples, gen_samples_mp4s

# ver = "10-st" # single trajectory ver
# ver = "test" # single trajectory ver
# ver = "11-st-run0"
# MAKE_SINGLE_CELL_TRAJ_SAMPLES = True
# DROP_MITOSIS_DIV = False

# ver = "10-drop-div"
# DROP_MITOSIS_DIV = True
# ver = "-test"

# ver = "11-drop-div"
# MAKE_SINGLE_CELL_TRAJ_SAMPLES = False
# DROP_MITOSIS_DIV = True

# ver = "12-st"
# MAKE_SINGLE_CELL_TRAJ_SAMPLES = True
# DROP_MITOSIS_DIV = False

# ver = "12-drop-div"
# MAKE_SINGLE_CELL_TRAJ_SAMPLES = False
# DROP_MITOSIS_DIV = True

ver = "12-all"
MAKE_SINGLE_CELL_TRAJ_SAMPLES = False
DROP_MITOSIS_DIV = False

In [17]:

data_dir = Path(f'notebook_results/mmaction_train_data_v{ver}')
class_labels = ['mitosis', 'apoptosis', 'normal']
class_label = "mitosis"
frame_types = ["video", "mask", "combined"]
fps = 3

# 1 instead of 0 to prevent the decord (used by mmdetection) python package error
padding_pixels = [1, 20, 40, 50, 100, 200, 400]



# split train and test data

# get #samples from all_class2samples
_split = 0.8

train_class2samples = {}
test_class2samples = {}
train_class2sample_extra_info = {}
test_class2sample_extra_info = {}

# randomize train and test data


for key in all_class2samples.keys():
    randomized_indices = np.random.permutation(len(all_class2samples[key])).astype(int)
    split_idx = int(len(all_class2samples[key]) * _split)
    _train_indices = randomized_indices[:split_idx]
    _test_indices = randomized_indices[split_idx:]
    train_class2samples[key] = np.array(all_class2samples[key], dtype=object)[_train_indices]
    test_class2samples[key] = np.array(all_class2samples[key], dtype=object)[_test_indices]

    train_class2sample_extra_info[key] = np.array(all_class2sample_extra_info[key], dtype=object)[_train_indices]
    test_class2sample_extra_info[key] = np.array(all_class2sample_extra_info[key], dtype=object)[_test_indices]



In [18]:
len(train_class2samples["normal"]), len(test_class2samples["normal"])

(1584, 396)

In [19]:
len(train_class2samples["mitosis"]), len(test_class2samples["mitosis"])

(158, 40)

In [20]:
video_frames_and_masks_from_sample(train_class2samples["normal"][6])[0][0].shape
# train_class2samples["normal"][6][1].show_panel()

(72, 97, 3)

In [21]:
import importlib
import livecellx
importlib.reload(livecellx.track.classify_utils)

<module 'livecellx.track.classify_utils' from '/home/ken67/LiveCellTracker-dev/livecellx/track/classify_utils.py'>

In [22]:
idx_to_check = 6
video_frames, video_frame_masks = video_frames_and_masks_from_sample(train_class2samples["normal"][idx_to_check], padding_pixels=0)
print("video frames dtype:", video_frames[0].dtype)
print("video frames shape:", video_frames[0].shape)
print("video frame masks dtype:", video_frame_masks[0].dtype)
print("video frame masks shape:", video_frame_masks[0].shape)
combined_frames = livecellx.track.classify_utils.combine_video_frames_and_masks(video_frames, video_frame_masks, edt_transform=True)
combined_frames = np.array(combined_frames).astype(np.uint8)
# combined_frames = np.maximum(combined_frames - 1, 0).astype(np.uint8)
print("combined_frames shape: ", combined_frames[0].shape)
gen_mp4_from_frames(combined_frames, "./test_video_output.mp4", fps=1)

video frames dtype: uint8
video frames shape: (72, 97, 3)
video frame masks dtype: uint8
video frame masks shape: (72, 97, 3)
combined_frames shape:  (72, 97, 3)


Visually check the generated frames' values

In [23]:
# channel = 2
# plt.imshow(combined_frames[0][..., channel])
# combined_frames[1][..., channel].max(), combined_frames[1][..., 0].shape

In [24]:
np.array(combined_frames).flatten().min()

0

Make single cell trajectories only (ONE cell per time frame)

In [25]:
from typing import Dict
from livecellx.track.data_prep_utils import check_one_sc_at_time
from livecellx.track.data_prep_utils import make_one_cell_per_timeframe_for_class2samples, make_one_cell_per_timeframe_helper, make_one_cell_per_timeframe_samples



sample = train_class2samples["mitosis"][0]

In [26]:
[sc.timeframe for sc in sample]

[127, 128, 129, 130, 131, 132, 133, 134, 134, 135, 135, 136, 136]

In [27]:
len(make_one_cell_per_timeframe_samples(sample))

8

In [28]:
if MAKE_SINGLE_CELL_TRAJ_SAMPLES:
    train_class2samples, train_class2sample_extra_info = make_one_cell_per_timeframe_for_class2samples(train_class2samples, train_class2sample_extra_info)
    test_class2samples, test_class2sample_extra_info = make_one_cell_per_timeframe_for_class2samples(test_class2samples, test_class2sample_extra_info)

Drop the cell divison part for easier inference durign testing

In [29]:
from livecellx.track.data_prep_utils import drop_multiple_cell_frames_in_samples

if DROP_MITOSIS_DIV:
    train_class2samples = drop_multiple_cell_frames_in_samples(train_class2samples)
    test_class2samples = drop_multiple_cell_frames_in_samples(test_class2samples)


In [30]:
for key, val in train_class2samples.items():
    assert len(val) == len(train_class2sample_extra_info[key]), f"key: {key}, len(val): {len(val)}, len(train_class2sample_extra_info[key]): {len(train_class2sample_extra_info[key])}"

In [31]:
# # for debug
# train_class2samples = {key: value[:5] for key, value in all_class2samples.items()}
# test_class2samples = {key: value[:5] for key, value in all_class2samples.items()}
# padding_pixels = [20]


train_sample_info_df = gen_class2sample_samples(
    train_class2samples,
    train_class2sample_extra_info,
    data_dir,
    class_labels,
    padding_pixels=padding_pixels,
    frame_types=frame_types,
    fps=fps,
    prefix="train",
)
test_sample_info_df = gen_class2sample_samples(
    test_class2samples,
    test_class2sample_extra_info,
    data_dir,
    class_labels,
    padding_pixels=padding_pixels,
    frame_types=frame_types,
    fps=fps,
    prefix="test",
)


|-----? No cells in the sample 33 with extra info {'src_dir': PosixPath('../datasets/mitosis-annotations-2023/shiman_CXA_high_density/C10^4')}, skipping...


100%|██████████| 157/157 [00:17<00:00,  9.07it/s]

|-----? No cells in the sample 33 with extra info {'src_dir': PosixPath('../datasets/mitosis-annotations-2023/shiman_CXA_high_density/C10^4')}, skipping...



100%|██████████| 157/157 [00:16<00:00,  9.64it/s]


|-----? No cells in the sample 33 with extra info {'src_dir': PosixPath('../datasets/mitosis-annotations-2023/shiman_CXA_high_density/C10^4')}, skipping...


...

100%|██████████| 157/157 [00:32<00:00,  4.84it/s]

|-----? No cells in the sample 33 with extra info {'src_dir': PosixPath('../datasets/mitosis-annotations-2023/shiman_CXA_high_density/C10^4')}, skipping...



100%|██████████| 157/157 [01:04<00:00,  2.42it/s]


|-----? No cells in the sample 33 with extra info {'src_dir': PosixPath('../datasets/mitosis-annotations-2023/shiman_CXA_high_density/C10^4')}, skipping...


100%|██████████| 157/157 [02:43<00:00,  1.04s/it]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 40/40 [00:43<00:00,  1.10s/it]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 396/396 [03:00<00:00,  2.20it/s]


In [32]:

train_sample_info_df.to_csv(
    data_dir / f"train_data.txt",
    index=False,
    header=True,
    sep=" ",
)
test_sample_info_df.to_csv(
    data_dir / f"test_data.txt",
    index=False,
    header=True,
    sep=" ",
)

mmaction_df_paths = []
for selected_frame_type in frame_types:
    train_df_path = data_dir / f"mmaction_train_data_{selected_frame_type}.txt"
    train_selected_frame_type_df = train_sample_info_df[train_sample_info_df["frame_type"] == selected_frame_type]
    train_selected_frame_type_df = train_selected_frame_type_df.reset_index(drop=True)
    train_selected_frame_type_df = train_selected_frame_type_df[["path", "label_index"]]
    train_selected_frame_type_df.to_csv(train_df_path, index=False, header=False, sep=" ")

    test_df_path = data_dir / f"mmaction_test_data_{selected_frame_type}.txt"
    test_selected_frame_type_df = test_sample_info_df[test_sample_info_df["frame_type"] == selected_frame_type]
    test_selected_frame_type_df = test_selected_frame_type_df[["path", "label_index"]]
    test_selected_frame_type_df = test_selected_frame_type_df.reset_index(drop=True)
    test_selected_frame_type_df.to_csv(test_df_path, index=False, header=False, sep=" ")

    mmaction_df_paths.append(train_df_path)
    mmaction_df_paths.append(test_df_path)


# # The follwing code generates v1-v7 test data. The issue is that some of test data shows up in train data, through different padding values.
# data_df_path = data_dir/'all_data.txt'
# sample_df = gen_samples_df(data_dir, class_labels, padding_pixels, frame_types, fps)
# sample_df.to_csv(data_df_path, index=False, header=False, sep=' ')
# for selected_frame_type in frame_types:
#     selected_frame_type_df = sample_df[sample_df["frame_type"] == selected_frame_type]
#     selected_frame_type_df = selected_frame_type_df.reset_index(drop=True)
#     train_df_path = data_dir/f'train_data_{selected_frame_type}.txt'
#     test_df_path = data_dir/f'test_data_{selected_frame_type}.txt'
#     train_df = selected_frame_type_df.sample(frac=0.8, random_state=0, replace=False)
#     test_df = selected_frame_type_df.drop(train_df.index, inplace=False)

#     # only keep the path and label_index columns
#     train_df = train_df[["path", "label_index"]]
#     test_df = test_df[["path", "label_index"]]

#     train_df.to_csv(train_df_path, index=False, header=False, sep=' ')
#     test_df.to_csv(test_df_path, index=False, header=False, sep=' ')


In [33]:
train_class2samples

{'mitosis': array([list([SingleCellStatic(id=5a688a30-c08f-4588-9e7e-cd5d6fb460a9, timeframe=127, bbox=[1468.  994. 1567. 1092.]), SingleCellStatic(id=1b18e13d-f90c-4d81-8e7b-ad07466623d5, timeframe=128, bbox=[1484.  999. 1569. 1096.]), SingleCellStatic(id=a8a38ba4-301d-46e6-af81-dac8de5fb9e3, timeframe=129, bbox=[1502. 1001. 1580. 1089.]), SingleCellStatic(id=284e598f-ed79-489e-9b68-a3caf9162b58, timeframe=130, bbox=[1507. 1001. 1584. 1087.]), SingleCellStatic(id=fb82574a-7354-4cf2-a9e9-6be97e86c0e3, timeframe=131, bbox=[1505. 1005. 1580. 1083.]), SingleCellStatic(id=055f94ab-91be-41d5-b2bc-2e05d602ae3f, timeframe=132, bbox=[1506. 1007. 1579. 1081.]), SingleCellStatic(id=6c51483f-ba59-4658-b43a-b695f0923da2, timeframe=133, bbox=[1507. 1001. 1572. 1080.]), SingleCellStatic(id=d7d57791-4f39-40cb-a773-718156e0b19b, timeframe=134, bbox=[1523. 1041. 1576. 1091.]), SingleCellStatic(id=8c95eb37-3a66-4757-af2b-495879af89a5, timeframe=134, bbox=[1488.  990. 1555. 1061.]), SingleCellStatic(id=9

Check the videos

In [34]:

video_paths = list(Path(data_dir/'videos').glob('*.mp4'))

Due to a `decord` package [issue](https://github.com/dmlc/decord/issues/150), to use mmaction2 we must check if the videos can be loaded by `decord` correctly.

In [35]:
import decord
decord.__version__

'0.6.0'

In [36]:
import decord
invalid_decord_paths = []
for path in video_paths:
# for path in ["./notebook_results/train_normal_6_raw_padding-0.mp4"]:
# for path in ["./test_video_output.mp4"]:
    reader = decord.VideoReader(str(path))
    reader.seek(0)
    imgs = list()
    frame_inds = range(0, len(reader))
    for idx in frame_inds:
        reader.seek(idx)
        frame = reader.next()
        imgs.append(frame.asnumpy())
        frame = frame.asnumpy()

        num_channels = frame.shape[-1]
        if num_channels != 3:
            print("invalid video for decord (https://github.com/dmlc/decord/issues/150): ", path)
            invalid_decord_paths.append(path)
            break
        # fig, axes = plt.subplots(1, num_channels, figsize=(20, 10))
        # for i in range(num_channels):
        #     axes[i].imshow(frame[:, :, i])
        # plt.show()
    del reader

invalid video for decord (https://github.com/dmlc/decord/issues/150):  notebook_results/mmaction_train_data_v12-all/videos/train_normal_1464_mask_padding-1.mp4
invalid video for decord (https://github.com/dmlc/decord/issues/150):  notebook_results/mmaction_train_data_v12-all/videos/train_normal_245_mask_padding-1.mp4
invalid video for decord (https://github.com/dmlc/decord/issues/150):  notebook_results/mmaction_train_data_v12-all/videos/train_normal_1131_raw_padding-1.mp4
invalid video for decord (https://github.com/dmlc/decord/issues/150):  notebook_results/mmaction_train_data_v12-all/videos/test_normal_365_mask_padding-1.mp4
invalid video for decord (https://github.com/dmlc/decord/issues/150):  notebook_results/mmaction_train_data_v12-all/videos/train_normal_1563_raw_padding-1.mp4
invalid video for decord (https://github.com/dmlc/decord/issues/150):  notebook_results/mmaction_train_data_v12-all/videos/test_normal_227_combined_padding-1.mp4
invalid video for decord (https://github.co

Remove "invalid" videos (cannot be read by decord) from mmdetection 

In [37]:
# extract file names from invalid decord paths
invalid_decord_filenames = set([os.path.basename(path) for path in invalid_decord_paths])

for df_path in mmaction_df_paths:
    _df = pd.read_csv(df_path, sep=" ", header=None)
    # remove all the rows with column "path" in invalid_decord_filenames
    filtered_df = _df[~_df[0].isin(invalid_decord_filenames)]

    df_filename = os.path.basename(df_path)
    # summarize the number of samples for the file
    print(f"df_path: {df_filename}, #filtered: {_df.shape[0] - filtered_df.shape[0]}, original df shape: {_df.shape}, filtered df shape: {filtered_df.shape}")

    # save to the disk
    filtered_df.to_csv(df_path, index=False, header=False, sep=" ")


df_path: mmaction_train_data_video.txt, #filtered: 5, original df shape: (12187, 2), filtered df shape: (12182, 2)
df_path: mmaction_test_data_video.txt, #filtered: 2, original df shape: (3052, 2), filtered df shape: (3050, 2)
df_path: mmaction_train_data_mask.txt, #filtered: 5, original df shape: (12187, 2), filtered df shape: (12182, 2)
df_path: mmaction_test_data_mask.txt, #filtered: 2, original df shape: (3052, 2), filtered df shape: (3050, 2)
df_path: mmaction_train_data_combined.txt, #filtered: 5, original df shape: (12187, 2), filtered df shape: (12182, 2)
df_path: mmaction_test_data_combined.txt, #filtered: 2, original df shape: (3052, 2), filtered df shape: (3050, 2)


check if videos can be loaded by cv2 correctly

In [38]:
import cv2

cap = cv2.VideoCapture("./test_video_output.mp4")

while True:
    ret, frame = cap.read()
    if not ret:
        break
    assert frame.shape[-1] == 3, "frame should be in RGB format"

cap.release()
cv2.destroyAllWindows()

In [39]:
# from sklearn.model_selection import train_test_split

# train_df_path = data_dir/'train_data.csv'
# test_df_path = data_dir/'test_data.csv'

# # split train and test from df
# train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
# train_df.to_csv(train_df_path, index=False, header=False, sep=' ')
# test_df.to_csv(test_df_path, index=False, header=False, sep=' ')
